<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>



### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://www.nvidia.com/en-sg/training/instructor-led-workshops/building-rag-agents-with-llms/). If sharing this material, please give credit and link back to the original course.

<br>


### **Environment Setup:**

In [1]:
# %%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

----

<br>

## Part 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/data_connection/)

----

<br>

## **Part 2:** RAG for Conversation History

In our previous explorations, we delved into the capabilities of document embedding models and used them to embed, store, and compare semantic vector representations of text. Though we could motivate how to efficiently extend this into vector store land manually, the true beauty of working with a standard API is its strong incorporation with other frameworks that can already do the heavy lifting for us!

<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [3]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>

### **Step 2:** Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

<br>

In addition to simplifying the process from an API perspective, vector stores also implement connectors, integrations, and optimizations under the hood. In our case, we will start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine!

**Specifically:**

1. We can feed our conversation into [**a FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss) via the `from_texts` constructor. This will take our conversational data and the embedding model to create a searchable index over our discussion.
2. This vector store can then be "interpreted" as a retriever, supporting the LangChain runnable API and returning documents retrieved via an input query.

The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [4]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 170 ms, sys: 330 ms, total: 500 ms
Wall time: 1.88 s


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [5]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [6]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content='[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard
many great things about them.'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

As we can see, our retriever found a handful of semantically relevant documents from our query. You may notice that not all of the documents are useful or clear on their own. For example, a retrieval of *"Beras"* for *"your name"* may be problematic for the chatbot if provided out of context. Anticipating the potential problems and creating synergies between your LLM components can increase the likelihood of good RAG behavior, so keep an eye out for such pitfalls and opportunities.

<br>

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [7]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [8]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

Based on the context provided, Beras lives in the Arctic. It's quite different from the warm climate of the Rocky 
Mountains!

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [10]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

The Rocky Mountains are a stunning mountain range that spans across North America. They are quite extensive and 
definitely worth looking into if you're curious! You can find plenty of information online or even in documentaries
to explore more about them.

In [11]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

The Rocky Mountains are a stunning mountain range that extends across North America. While they don't directly 
border California, they do run through parts of the western United States. If you're ever interested in visiting, 
California is a great starting point for a road trip up north!

In [12]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

Based on the information provided, it's not specified how far Beras is from the Rocky Mountains. However, we do 
know that Beras lives in the Arctic and is not used to the warm climate of the Rocky Mountains, which suggests a 
significant distance.

<br>

You might notice some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

Oh, I'm sorry for the confusion, but the Rocky Mountains aren't particularly known for ice cream, they're actually 
famous for their breathtaking scenery and diverse wildlife. However, I'm sure you can find some delicious ice cream
nearby! It's always a good treat, especially after a long day of exploring the mountains.

Given our conversation, I'd guess that you're a big fan of ice cream! You seem to be particularly excited about 
enjoying it after exploring the Rocky Mountains. Would you like to learn more about the mountains, or perhaps 
discuss different ice cream flavors?

I see! I apologize for the misunderstanding earlier. My guess was based on your enthusiasm for ice cream in your 
previous messages. However, honey is a wonderful choice as well! It's such a versatile sweetener and has some 
amazing health benefits too. Is there anything specific about honey that you love?

I sure do! Despite the initial confusion, I can confidently say that your favorite food is indeed ice cream! Though
you have a love for honey as well, your excitement about ice cream was clear from your previous messages! It's 
always a delight to chat about delicious food, isn't it?

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## **Part 3 [Exercise]:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!

<br>

### **Exercise:**

In the previous example, you may recall that we pulled in some relatively small papers with the help of [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) using the following syntax:

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

Given all that you've learned so far, choose a selection of papers that you would like to use and develop a chatbot that can talk about them!

<br>

Though this is a pretty big task, a walkthrough of ***most*** of the process will be provided below. By the end of the walkthrough, many of the necessary puzzle pieces will be provided, and your real task will be to integrate them together for the final `retrieval_chain`. When you're done, get ready to re-integrate the chain (or a flavor of your choice) in the last notebook as part of the evaluation exercise!


<br>

### **Task 1**: Loading And Chunking Your Documents

The following code block gives you some default papers to load in for your RAG chain. Feel free to select more papers as desired, but note that longer documents will take longer to process. A few simplifying assumptions and additional processing steps are included to help you improve your naive RAG performance:

- Documents are cut off prior to the "References" section if one exists. This will keep our system from considering the citations and appendix sections, which tend to be long and distracting.

- A chunk that lists the available documents is inserted to provide a high-level view of all available documents in a single chunk. If your pipeline does not provide metadata on each retrieval, this is a useful component and can even be listed among a list of higher-priority pieces if appropriate.

- Additionally, the metadata entries are also inserted to provide general information. Ideally, there would also be some synthetic chunks that merge the metadata into interesting cross-document chunks.

**NOTE:** ***For the sake of the assessment, please include at least one paper that is less than one month old!***


In [14]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    #ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    #ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    #ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    #ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    #ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    #ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ## Some longer papers
    # ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    # ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    # ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more

    ArxivLoader(query="2408.02339").load(),  ## Modeling the impact of Climate transition on real estate prices
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Modeling the impact of Climate transition on real estate prices 

Document 0
 - # Chunks: 62
 - Metadata: 


{
    'Published': '2024-08-05',
    'Title': 'Modeling the impact of Climate transition on real estate prices',
    'Authors': 'Lionel Sopgoui',
    'Summary': 'In this work, we propose a model to quantify the impact of the climate\ntransition on a property in
housing market. We begin by noting that property is\nan asset in an economy. That economy is organized in sectors, 
driven by its\nproductivity which is a multidimensional Ornstein-Uhlenbeck process, while the\nclimate transition 
is declined thanks to the carbon price, a continuous\ndeterministic process. We then extend the sales comparison 
approach and the\nincome approach to valuate an energy inefficient real estate asset. We obtain\nits value as the 
difference between the price of an equivalent efficient\nbuilding following an exponential Ornstein-Uhlenbeck as 
well as the actualized\nrenovation costs and the actualized sum of the future additional energy costs.\nThese costs
are due to the inefficiency of the building, before an optimal\nrenovation date which depends on the carbon price 
process. Finally, we carry\nout simulations based on the French economy and the house price index of\nFrance. Our 
results allow to conclude that the order of magnitude of the\ndepreciation obtained by our model is the same as the
empirical observations.'
}

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [15]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 65 ms, sys: 17.1 ms, total: 82.1 ms
Wall time: 6.36 s


<br>

From there, we can combine our indices into a single one using the following utility:

In [16]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 64 chunks


<br>

### **Task 3: [Exercise]** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [19]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

################################################################################################

retrieval_chain = (
    {'input' : (lambda x: x)}
    ##  Make sure to retrieve history & context from convstore & docstore, respectively.
   
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str}) 
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str})
)


################################################################################################

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Is the climate affecting real estate prices!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Is the climate affecting real estate prices!\n\n From this, we have retrieved the following 
potentially-useful info:  Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from Document] 
Available Documents:\n - Modeling the impact of Climate transition on real estate prices\n[Quote from Modeling the 
impact of Climate transition on real estate prices] . Aydin et al. (2020) found that EE is capitalized quite 
precisely\\ninto home prices in the Dutch housing market. de Ayala et al. (2016) found in a study on 1507\\nhomes 
in Spain that dweelings labelled A, B or C are valued at between 5.4% and 9.8% higher price\\ncompared to D, E, F 
or G rated home. Franke and Nadler (2019) also highlight that, in the rental\\ndecision-making, EE achieves a high 
importance score similar to that of rent, price and location\\nrespectively.\\nThere is a great deal of statistical
work on the effect of climate change on real estate, but very\\n1in ton of CO2 emissions per square meter per year 
or kilowatt hour per square meter per year\\nPage 3\\nModeling the impact of Climate transition on real estate 
prices\\nlittle modelling. It is this last point that interests us here\n[Quote from Modeling the impact of Climate
transition on real estate prices] . If we look at Figure 4, we remark that when the valuation date is later 
than\\nthe optimal renovation date, the best thing to do is to renovate directly. This stabilizes or 
even\\nreverses the price decline. Moreover, by adding the energy costs before renovations, we could reach\\n20 to 
30% of depreciation when the carbon price (so the energy price) is pretty high. This seems\\nPage 20\\nModeling the
impact of Climate transition on real estate prices\\nconsistent with the idea that a thermal sieve loses all its 
value and could become impossible to sell,\\nbecause of the enormous costs involved in owning it.\\nFigure 4: 
Apartment value slowdown per scenario and per year\\nFinally, it should be noted that the property value discounts 
obtained in this work correspond\\nto statistical observations, for example by de Ayala et al. 
(2016).\\nConclusion\\nThe aim of this work was to propose a model to assess and quantify the impact of the 
climate\\ntransition on real estate prices\n[Quote from Modeling the impact of Climate transition on real estate 
prices] "Modeling the impact of Climate transition on real estate prices\\nLionel Sopgouia,b,c\\na Laboratoire de 
Probabilit\\u00b4\\nes, Statistique et Mod\\u00b4\\nelisation (LPSM), Universit\\u00b4\\ne Paris 
Cit\\u00b4\\ne\\nbDepartment of Mathematics, Imperial College London\\ncValidation des mod`\\neles, Direction des 
risques, Groupe BPCE\\nAbstract\\nIn this work, we propose a model to quantify the impact of the climate transition
on a property\\nin housing market. We begin by noting that property is an asset in an economy. That economy\\nis 
organized in sectors, driven by its productivity which is a multidimensional Ornstein-Uhlenbeck\\nprocess, while 
the climate transition is declined thanks to the carbon price, a continuous deterministic\\nprocess. We then extend
the sales comparison approach and the income approach to valuate an\\nenergy inefficient real estate asset\n\n\n 
(Answer only from retrieval. Only cite sources that are used. Make your response conversational.)'
        ),
        HumanMessage(content='Is the climate affecting real estate prices!')
    ]
)

Yes, the climate is affecting real estate prices. According to a document titled "Modeling the impact of Climate transition on real estate prices," studies have shown that energy efficiency (EE) is capitalized into home prices. For example, in the Dutch housing market, homes labeled A, B, or C are valued at between 5.4% and 9.8% higher prices compared to D, E, F, or G rated homes. Similarly, in Spain, homes with better energy efficiency ratings are valued higher.

Furthermore, the study suggests that as the carbon price (energy price) increases, the depreciation of properties with high energy costs also increases. This is consistent with the idea that a thermal sieve (a property with poor energy efficiency) may lose all its value and become impossible to sell due to the enormous costs involved in owning it.

The aim of the study was to propose a model to assess and quantify the impact of the climate transition on real estate prices. Therefore, it can be inferred that the climate transi

### **Task 4:** Interact With Your Gradio Chatbot

In [20]:
 chatbot = gr.Chatbot(value = [[None, initial_msg]])
 demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

 try:
     demo.launch(debug=True, share=True, show_api=False)
     demo.close()
 except Exception as e:
     demo.close()
     print(e)
     raise e

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://6f2fd2437608ad5bc9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: To what extent will climate change affect depreciation?\n\n From this, we have retrieved the following 
potentially-useful info:  Conversation History Retrieval:\n[Quote from Document] User previously responded with Is 
the climate affecting real estate prices!\n[Quote from Document] Agent previously responded with Yes, the climate 
is affecting real estate prices. According to a document titled "Modeling the impact of Climate transition on real 
estate prices," studies have shown that energy efficiency (EE) is capitalized into home prices. For example, in the
Dutch housing market, homes labeled A, B, or C are valued at between 5.4% and 9.8% higher prices compared to D, E, 
F, or G rated homes. Similarly, in Spain, homes with better energy efficiency ratings are valued 
higher.\n\nFurthermore, the study suggests that as the carbon price (energy price) increases, the depreciation of 
properties with high energy costs also increases. This is consistent with the idea that a thermal sieve (a property
with poor energy efficiency) may lose all its value and become impossible to sell due to the enormous costs 
involved in owning it.\n\nThe aim of the study was to propose a model to assess and quantify the impact of the 
climate transition on real estate prices. Therefore, it can be inferred that the climate transition has an impact 
on real estate prices.\n\nSource: Modeling the impact of Climate transition on real estate prices. Available at: 
[Quote from Modeling the impact of Climate transition on real estate prices].\n\n\n Document Retrieval:\n[Quote 
from Document] {\'Published\': \'2024-08-05\', \'Title\': \'Modeling the impact of Climate transition on real 
estate prices\', \'Authors\': \'Lionel Sopgoui\', \'Summary\': \'In this work, we propose a model to quantify the 
impact of the climate\\ntransition on a property in housing market. We begin by noting that property is\\nan asset 
in an economy. That economy is organized in sectors, driven by its\\nproductivity which is a multidimensional 
Ornstein-Uhlenbeck process, while the\\nclimate transition is declined thanks to the carbon price, a 
continuous\\ndeterministic process. We then extend the sales comparison approach and the\\nincome approach to 
valuate an energy inefficient real estate asset. We obtain\\nits value as the difference between the price of an 
equivalent efficient\\nbuilding following an exponential Ornstein-Uhlenbeck as well as the actualized\\nrenovation 
costs and the actualized sum of the future additional energy costs.\\nThese costs are due to the inefficiency of 
the building, before an optimal\\nrenovation date which depends on the carbon price process. Finally, we 
carry\\nout simulations based on the French economy and the house price index of\\nFrance. Our results allow to 
conclude that the order of magnitude of the\\ndepreciation obtained by our model is the same as the empirical 
observations.\'}\n[Quote from Modeling the impact of Climate transition on real estate prices] . For the income 
approach, the expected net (imputed)\\nincome is the tricky quantity to determine. Schulz (2003) takes into account
the relative maintenance\\nand repair expenditures, the depreciating rate, the real estate tax rate, the real 
interest rate, the\\ninflation of the rent (or imputed rent \\u2013 the rental price an individual would pay for an
asset they\\nown \\u2013), etc. We will use these two approaches in this work.\\nClimate change has a deep impact 
on human societies and their environments.\\nOne of the\\ncomponents of the climate risk is transition risk which 
relates to the potential economic and financial\\nlosses associated with the process of adjusting towards a 
low-carbon economy.\\nTransition risk\\nis becoming increasingly important in all parts of the economy and 

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Can anything be done to stop the deprecation of real estate due to climate change \n\n From this, we 
have retrieved the following potentially-useful info:  Conversation History Retrieval:\n[Quote from Document] Agent
previously responded with Yes, the climate is affecting real estate prices. According to a document titled 
"Modeling the impact of Climate transition on real estate prices," studies have shown that energy efficiency (EE) 
is capitalized into home prices. For example, in the Dutch housing market, homes labeled A, B, or C are valued at 
between 5.4% and 9.8% higher prices compared to D, E, F, or G rated homes. Similarly, in Spain, homes with better 
energy efficiency ratings are valued higher.\n\nFurthermore, the study suggests that as the carbon price (energy 
price) increases, the depreciation of properties with high energy costs also increases. This is consistent with the
idea that a thermal sieve (a property with poor energy efficiency) may lose all its value and become impossible to 
sell due to the enormous costs involved in owning it.\n\nThe aim of the study was to propose a model to assess and 
quantify the impact of the climate transition on real estate prices. Therefore, it can be inferred that the climate
transition has an impact on real estate prices.\n\nSource: Modeling the impact of Climate transition on real estate
prices. Available at: [Quote from Modeling the impact of Climate transition on real estate prices].\n[Quote from 
Document] User previously responded with Is the climate affecting real estate prices!\n[Quote from Document] User 
previously responded with To what extent will climate change affect depreciation?\n[Quote from Document] Agent 
previously responded with Climate change and the transition to a low-carbon economy can have an impact on the 
depreciation of real estate, particularly properties with high energy costs. According to a document titled 
"Modeling the impact of Climate transition on real estate prices," as the carbon price (which is a measure of the 
energy price) increases, the depreciation of these properties also increases. This is consistent with the idea that
a property with poor energy efficiency, also known as a "thermal sieve," may lose all its value and become 
impossible to sell due to the high costs involved in owning it. The study also suggests that renovating a property 
before its optimal renovation date can stabilize or even reverse the price decline. These findings are based on 
simulations using the French economy and house price index as examples.\n\nSource: Modeling the impact of Climate 
transition on real estate prices. Available at: [Quote from Modeling the impact of Climate transition on real 
estate prices].\n\n\n Document Retrieval:\n[Quote from Modeling the impact of Climate transition on real estate 
prices] "Modeling the impact of Climate transition on real estate prices\\nLionel Sopgouia,b,c\\na Laboratoire de 
Probabilit\\u00b4\\nes, Statistique et Mod\\u00b4\\nelisation (LPSM), Universit\\u00b4\\ne Paris 
Cit\\u00b4\\ne\\nbDepartment of Mathematics, Imperial College London\\ncValidation des mod`\\neles, Direction des 
risques, Groupe BPCE\\nAbstract\\nIn this work, we propose a model to quantify the impact of the climate transition
on a property\\nin housing market. We begin by noting that property is an asset in an economy. That economy\\nis 
organized in sectors, driven by its productivity which is a multidimensional Ornstein-Uhlenbeck\\nprocess, while 
the climate transition is declined thanks to the carbon price, a continuous deterministic\\nprocess. We then extend
the sales comparison approach and the income approach to valuate an\\nenergy inefficient real estate asset\n[Quote 
from Modeling the impact of Climate transition on real estate prices] . The fram

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6f2fd2437608ad5bc9.gradio.live
Closing server running on port: 7860


<br>

----

<br>

## **Part 4:** Saving Your Index For Evaluation

After you've implemented your RAG chain, please save your accumulated vector store as shown [in the official documentation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading). You'll have a chance to use it again for your final assessment!

In [21]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed). After you have confirmed that the cell can pull in your index, download `docstore_index.tgz` for use in the last notebook!

In [22]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
.013\n0.208\n0.039\n0.037\nLow\n0.072\n0.039\n0.059\n0.020\nVery Low\n0.092\n0.037\n0.020\n0.068\nTable 4: Parameter \u03a3 of the productivity\nPage 18\nModeling the impact of Climate transition on real estate prices\n4.3.2. The housing pricing index (HPI)\nWe write the housing price index K in Base 2021 and we apply the logarithm function. This\nmeans that Kt0 = 0 as shown in Figure 2. We can therefore calibrate \u03f1, \u03d1, \u03bd, \u03c3, and \u03c1. The values\nare presented in Table 5 below.\nParameter\nValue\n\u03f1\n0.024\n\u03d1\n-0.884\n\u03bd\n0.026\n\u03c3\n0.050\n\u03c1\n[-0.019, -0.042, -0.017, 0.015]\nTable 5: Housing price index parameters\n4.4


-----

<br>

## **Part 5:** Wrap-Up

Congratulations! Assuming your RAG chain is all good, you're now ready to move on to the **RAG Evaluation [Assessment]** section!


<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>